In [22]:
# !pip install h2o

In [23]:
import pandas as pd

import h2o
from h2o.automl import H2OAutoML

import json
import os
import re

In [24]:
'''
Para executar o autoML, favor alterar os campos source_path, saving_path com os diretórios onde os 
dados de origem para treinamento e o lugar onde serão salvos os modelos treinados, alterar o target com a 
coluna que se tem como objetivo. O exclude_algos são os algoritmos que serão excluidos do treinamento, 
acima do mesmo encontra-se a lista de algoritmos disponíveis para o h2o. Max models é o numero máximo de modelos 
que serão treinados e comparados, seed para auxiliar na reprodução dos treinamentos, max_training_time é
o tempo máximo para treinamento e classification é para selecionar se é um treinamento de classificação ou reressão.
O treinamento para classificação não fora concluído, apenas o de regressão.
'''

source_path = 'https://s3model.blob.core.windows.net/prodata/merged_data.csv'
saving_path = 'https://s3model.blob.core.windows.net/modeldata/'

target = 'average_ticket'

# Available algorithms: ['DRF', 'GLM', 'XGBoost', 'GBM', 'DeepLearning', 'StackedEnsemble']
exclude_algos = ["DeepLearning"]

max_models = 5
seed = 23
max_training_time = 60
classification = False 

In [25]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 21 mins
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,"28 days, 5 hours and 24 minutes"
H2O_cluster_name:,H2O_from_python_hobbes_y4kzmu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.519 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [26]:
df = h2o.import_file(path=source_path)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [27]:
train, test = df.split_frame(ratios=[0.8], seed = seed)

In [28]:
# include_algos = ['xgboost']
aml = H2OAutoML(max_models = max_models, max_runtime_secs = max_training_time, exclude_algos = exclude_algos, balance_classes=True, seed = seed)

In [29]:
aml.train(training_frame = train, y = target)

AutoML progress: |
20:22:16.245: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

██████████████████
20:22:35.310: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

███████████████████████████████████████
20:23:11.504: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

██████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220118_202215


Model Summary: 


,,number_of_trees
0,,13.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 11.51907312584116
RMSE: 3.3939760054898973
MAE: 1.941909033232072
RMSLE: 0.061399994035527435
Mean Residual Deviance: 11.51907312584116

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 17.531404387664487
RMSE: 4.187051992472089
MAE: 2.4711242135490137
RMSLE: 0.07561767788069695
Mean Residual Deviance: 17.531404387664487

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,2.471121,0.235907,2.578224,2.803442,2.404905,2.400002,2.169031
1,mean_residual_deviance,17.531351,2.886981,19.291239,21.576204,16.277002,16.244621,14.267683
2,mse,17.531351,2.886981,19.291239,21.576204,16.277002,16.244621,14.267683
3,r2,0.950747,0.007957,0.945450,0.939783,0.954715,0.954281,0.959507
4,residual_deviance,17.531351,2.886981,19.291239,21.576204,16.277002,16.244621,14.267683
5,rmse,4.175879,0.341657,4.392179,4.645020,4.034477,4.030462,3.777259
6,rmsle,0.075507,0.004580,0.077510,0.082480,0.073448,0.073289,0.070805



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-01-18 21:22:31,15.161 sec,0.0,58.439122,55.311031,3415.130971
1,,2022-01-18 21:22:33,16.801 sec,5.0,11.350070,9.852360,128.824090
2,,2022-01-18 21:22:34,17.898 sec,10.0,4.117752,2.721086,16.955879
3,,2022-01-18 21:22:34,18.670 sec,13.0,3.393976,1.941909,11.519073



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,price_range,1.075888e+07,1.000000,0.834939
1,minimum_order_value,4.195903e+05,0.038999,0.032562
2,order_total_amount,3.100477e+05,0.028818,0.024061
3,merchant_zip_code,2.181958e+05,0.020281,0.016933
4,delivery_address_longitude,1.293045e+05,0.012018,0.010035
5,enabled.true,1.085515e+05,0.010089,0.008424
6,delivery_time,1.072228e+05,0.009966,0.008321
7,takeout_time,1.072048e+05,0.009964,0.008320
8,delivery_address_latitude,9.570969e+04,0.008896,0.007428
9,enabled.false,7.673938e+04,0.007133,0.005955



See the whole table with table.as_data_frame()


In [30]:
best_model = aml.get_best_model()

In [31]:
model_info = best_model.model_performance(test)

In [32]:
model_path_1 = h2o.save_model(model=best_model,path=saving_path, force=True)

In [33]:
best_model = model_path_1.split('/')[int(len(model_path_1.split('/'))-1)]

In [34]:
model_id = best_model.split('_')[-2]+best_model.split('_')[-1]
model_name = best_model.split('_')[0]+'_'+model_id
model_date = best_model.split('_')[-2]

In [35]:
test_path = saving_path + '/test_' + model_id+ '.csv'
train_path = saving_path + '/train_' + model_id +'.csv'

h2o.export_file(test, path = test_path , force = True)
h2o.export_file(train, path = train_path , force = True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [36]:
model_path = saving_path +'/'+ model_name

try:
    aic = model_info.aic()
except:
    aic = str(0)

if classification == False:
    info = {
        'id': str(model_id),
        'name': str(model_name),
        'data': str(model_date),
        "h2o_version": str(h2o.__version__),
        "seed": str(seed),
        "target": str(target),
        'mse':str(model_info.mse()),
        'rmse':str(model_info.rmse()),
        'mae':str(model_info.mae()),
        'rmsle':str(model_info.rmsle()),
        'r2':str(model_info.r2()),
        'mean_resid_deviance':str(model_info.mean_residual_deviance()),
        'null_degrees_of_freedom':str(model_info.null_degrees_of_freedom()),
        'resid_degrees_of_freedom':str(model_info.residual_degrees_of_freedom()),
        'null_deviance':str(model_info.null_deviance()),
        'residual_deviance':str(model_info.residual_deviance()),
        'aic':aic,
        "train_path": str(train_path),
        "test_path": str(test_path),
        'model_path':str(model_path)
    }
    
# elif classification == True:
#     info = {
#         'id': model_id,
#         'name': model_name,
#         'data': model_date,
#         "h2o_version": h2o.__version__,
#         "seed": str(seed),
#         "target": target,
#         "train_data": train_path,
#         "test_data": test_path,
#         'mse':model_info.mse(),
#         'rmse':model_info.rmse(),
#         'mae':model_info.mae(),
#         'rmsle':model_info.rmsle()
#         'r2':model_info.r2(),
#         'mean_resid_deviance':model_info.mean_residual_deviance(),
#         'null_degrees_of_freedom()':model_info.null_degrees_of_freedom(),
#         'resid_degrees_of_freedom':model_info.residual_degrees_of_freedom(),
        
#         'aic':model_info.aic()
        
#     }

In [37]:
info_new = pd.DataFrame([info])

info_path = saving_path + '/' + 'info.csv'

try:
    info_old = pd.read_csv(info_path, index_col=[0])
    info_save = pd.concat([info_old, info_new])
    info_save.to_csv(info_path)
except:
    info_new.to_csv(info_path)

In [50]:
info_old 

,id,name,data,h2o_version,seed,target,mse,rmse,mae,rmsle,r2,mean_resid_deviance,null_degrees_of_freedom,resid_degrees_of_freedom,null_deviance,residual_deviance,aic,train_path,test_path,model_path
0,20220112154630,StackedEnsemble_20220112154630,20220112,3.34.0.7,1412,average_ticket,5.181914,2.276382,1.058793,0.042543,0.985469,5.181914,7986,7983,2848984.895475833,41387.94397116301,0,/home/hobbes/Downloads/IFood/data/model/train_20220112154630.csv,/home/hobbes/Downloads/IFood/data/model/test_20220112154630.csv,/home/hobbes/Downloads/IFood/data/model/StackedEnsemble_2022011215...
0,20220112155358,XGBoost_20220112155358,20220112,3.34.0.7,22,average_ticket,13.582130,3.685394,2.430961,0.070916,0.961408,13.582130,None,None,None,None,0,/home/hobbes/Downloads/IFood/data/model/train_20220112155358.csv,/home/hobbes/Downloads/IFood/data/model/test_20220112155358.csv,/home/hobbes/Downloads/IFood/data/model/XGBoost_20220112155358
0,20220118202215,XGBoost_20220118202215,20220118,3.34.0.7,23,average_ticket,16.819745,4.101188,2.316419,0.075361,0.952429,16.819745,None,None,None,None,0,/home/hobbes/Downloads/IFood/data/model//train_20220118202215.csv,/home/hobbes/Downloads/IFood/data/model//test_20220118202215.csv,/home/hobbes/Downloads/IFood/data/model//XGBoost_20220118202215


In [38]:
os.rename(model_path_1, saving_path + '/'+ model_name)

In [47]:
load_path = model_path
loaded_model = h2o.load_model(path='/home/hobbes/Downloads/IFood/data/model/XGBoost_20220118202215')

In [48]:
preds = loaded_model.predict(test).as_data_frame()
preds['target'] = test[target].as_data_frame()

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [49]:
display(preds)

,predict,target
0,76.966217,80
1,76.966217,80
2,29.632746,30
3,59.606659,60
4,60.625885,60
...,...,...
7993,28.832254,30
7994,28.832254,30
7995,60.865658,60
7996,57.619953,60


In [42]:
info_old = pd.read_csv(info_path, index_col=[0])

In [43]:
load_path

'/home/hobbes/Downloads/IFood/data/model//XGBoost_20220118202215'